In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PyTorch Multi-Node Distributed Training on Vertex Training w/ Custom Container

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/brianchunkang/ai-samples/blob/master/pytorch_image_classification_torchrun_ddp_training_sdk/multi_node_torchrun_ddp_vertex_training_custom_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/brianchunkang/ai-samples/blob/master/pytorch_image_classification_torchrun_ddp_training_sdk/multi_node_torchrun_ddp_vertex_training_custom_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/brianchunkang/ai-samples/blob/master/pytorch_image_classification_torchrun_ddp_training_sdk/multi_node_torchrun_ddp_vertex_training_custom_container.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This tutorial uses the ImageNet dataset to run multi-node distributed training on Vertex AI with torch.multiprocessing. It will run a CPU only training and a distributed nodes with GPU training. The Tiny ImageNet (230MB in size) dataset is used

### Dataset

Download the Imagenet data here. You'll need to sign up for an Imagenet account: https://image-net.org/download-images.php

For the sake of training time, the Tiny ImageNet dataset is used in this tutorial: https://image-net.org/data/tiny-imagenet-200.zip

The training code is based on this PyTorch Torchrun example for ImageNet, updated to use torch.multiprocessing: https://github.com/pytorch/elastic/blob/master/examples/imagenet/main.py

### Objective

In this notebook, you will learn how to train an Imagenet model using PyTorch's torch.multiprocessing on multiple nodes:

    * Install necessary libraries
    * Download data to Google Cloud Storage
    * Create custom containers to train the model using code from PyTorch Elastic's Github repository
    * Train the model using multiple nodes using GPUs
    
### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as {XGBoost, AdaNet, or TensorFlow Hub TODO: Replace with relevant packages for the tutorial}. Use the latest major GA version of each package.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip install -U {USER_FLAG} google-cloud-aiplatform

In [ ]:
! pip install -U {USER_FLAG} torch==1.11.0
! pip install -U {USER_FLAG} torchvision==0.12.0

### Restart the kernel

Once you've installed everything, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

In this tutorial, Vertex AI pulls the data to a Cloud Storage location, and saves the
trained model that results from your job in the same bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. We suggest that you [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions).

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}
DATA_BUCKET_NAME='[YOUR-DATA-BUCKET-NAME]' # Can be the name of the same bucket above

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

if REGION == "[your-region]":
    REGION = "us-central1"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Service Account

You use a service account to create the Vertex AI Training job. If you do not want to use your project's Compute Engine service account, set SERVICE_ACCOUNT to another service account ID.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if IS_GOOGLE_CLOUD_NOTEBOOK:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if not IS_GOOGLE_CLOUD_NOTEBOOK:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
    
    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Enable Artifact Registry API

First, you must enable the Artifact Registry API service for your project.

Learn more about [Enabling service](https://cloud.google.com/artifact-registry/docs/enable-service).

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

### Create a private Docker repository

Your first step is to create your own Docker repository in Google Artifact Registry.

1. Run the `gcloud artifacts repositories create` command to create a new Docker repository with your region with the description "docker repository".

2. Run the `gcloud artifacts repositories list` command to verify that your repository was created.

In [ ]:
REPOSITORY = "pytorch-ddp-imagenet-repo"

In [ ]:
! gcloud artifacts repositories create {REPOSITORY} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### Configure authentication to your private repo

Before you push or pull container images, configure Docker to use the `gcloud` command-line tool to authenticate requests to `Artifact Registry` for your region.

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

## Vertex Training using Vertex SDK and Custom Container

### Initialize Vertex SDK

In [ ]:
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_URI,
    location=REGION,
)

### Create a Vertex Tensorboard Instance

In [ ]:
content_name = "pytorch-imagenet-multiprocessing-ddp"

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=content_name,
)

#### Option: Use a Previously Created Vertex Tensorboard Instance

```
tensorboard_name = "Your Tensorboard Resource Name or Tensorboard ID"
tensorboard = aiplatform.Tensorboard(tensorboard_name=tensorboard_name)
```

In [ ]:
TENSORBOARD_NAME = tensorboard.resource_name
print (TENSORBOARD_NAME)

## Vertex Training with GPUs

### Build Custom Container for GPUs

In [ ]:
CONTAINER_NAME = content_name + '-gpu'
TAG = "latest"

custom_container_image_uri = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{CONTAINER_NAME}:{TAG}"

In [ ]:
print (custom_container_image_uri)

In [ ]:
%cd trainer-gpu

### Test locally - data must be in a /gcs/$DATA_BUCKET_NAME/tiny-imagenet-200 folder

In [ ]:
import os
#Set env for local test
os.environ["MASTER_ADDR"]="127.0.0.1"
os.environ["MASTER_PORT"]="8081"
os.environ["RANK"]="0"
os.environ['WORLD_SIZE']="-1"

# Set to local test values
EPOCHS=1
BATCH_SIZE=32
NUM_CPUS=4

In [ ]:
! rm /tmp/checkpoint.pth.tar
! python main.py --epochs $EPOCHS --arch mobilenet_v3_large --batch-size $BATCH_SIZE --dist-backend nccl \
--data /gcs/$DATA_BUCKET_NAME/tiny-imagenet-200 --workers $NUM_CPUS --ngpus-per-node 1
! rm -r /tmp/gcs

### Build Docker container

In [ ]:
! docker build -t $custom_container_image_uri -f Dockerfile .

In [ ]:
!docker stop local-ts
!docker rm local-ts
!docker run --gpus all -it --rm \
--shm-size=2gb \
--name=local-ts \
--mount "type=volume,dst=/gcs,volume-driver=local,volume-opt=type=none,volume-opt=o=bind,volume-opt=device=/gcs" \
-e MASTER_ADDR="127.0.0.1" \
-e MASTER_PORT=29500 \
$custom_container_image_uri \
--epochs $EPOCHS --arch mobilenet_v3_large --batch-size $BATCH_SIZE --dist-backend nccl \
--data /gcs/$DATA_BUCKET_NAME/tiny-imagenet-200 \
--workers $NUM_CPUS \
--ngpus-per-node 1

In [ ]:
! docker push $custom_container_image_uri

In [ ]:
! gcloud artifacts docker images list $REGION-docker.pkg.dev/$PROJECT_ID/$REPOSITORY

In [ ]:
%cd ..

### Run a Vertex SDK CustomJob with GPUs

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
PRIMARY_COMPUTE = "n1-highmem-16"
TRAIN_COMPUTE = "n1-highmem-16"
NUM_CPUS = 8 #Set to a few less than max CPUs per instance up to 8 per recommendation from PyTorch
TRAIN_GPU = 'NVIDIA_TESLA_T4'
TRAIN_NGPU = 1
BATCH_SIZE = 256
REPLICAS = 5
EPOCHS = 100

display_name = CONTAINER_NAME + f'{REPLICAS}workers-{TRAIN_NGPU}{TRAIN_GPU}-{BATCH_SIZE}batch-' + TIMESTAMP
gcs_output_uri_prefix = f"{BUCKET_URI}/{display_name}"

CMDARGS = [
    f"--epochs={EPOCHS}",
    "--arch=mobilenet_v3_large",
    f"--batch-size={BATCH_SIZE}",
    "--dist-backend=nccl",
    f"--data=/gcs/{DATA_BUCKET_NAME}/tiny-imagenet-200",
    f"--workers={NUM_CPUS}",
    f"--ngpus-per-node={TRAIN_NGPU}"
]

CONTAINER_SPEC = {"image_uri": custom_container_image_uri, 
                  "args": CMDARGS}

PRIMARY_WORKER_POOL = {
    "replica_count": 1,
    "machine_spec": {"machine_type": PRIMARY_COMPUTE, "accelerator_count": TRAIN_NGPU, "accelerator_type": TRAIN_GPU,},
    "container_spec": CONTAINER_SPEC,
}

WORKER_POOL_SPECS = [PRIMARY_WORKER_POOL]

TRAIN_WORKER_POOL = {
    "replica_count": REPLICAS,
    "machine_spec": {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_count": TRAIN_NGPU,
        "accelerator_type": TRAIN_GPU,
    },
    "container_spec": CONTAINER_SPEC,
}

WORKER_POOL_SPECS.append(TRAIN_WORKER_POOL)

job = aiplatform.CustomJob(
    display_name=display_name, 
    base_output_dir=gcs_output_uri_prefix,
    worker_pool_specs=WORKER_POOL_SPECS
)

In [ ]:
job.run(sync=True,
       tensorboard=tensorboard.resource_name,
       service_account=SERVICE_ACCOUNT,
       enable_web_access=True,)

### For easier debugging in Vertex: Train using Custom Training Job and main.py script

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
PRIMARY_COMPUTE = "n1-highmem-16"
TRAIN_COMPUTE = "n1-highmem-16"
NUM_CPUS = 8 #Set to a few less than max CPUs per instance up to 8 per recommendation from PyTorch
TRAIN_GPU = 'NVIDIA_TESLA_T4'
TRAIN_NGPU = 1
BATCH_SIZE = 64
REPLICAS = 2
EPOCHS = 7

display_name = f'PyTorch-training-with-script-{REPLICAS}workers-{TRAIN_NGPU}{TRAIN_GPU}-' + TIMESTAMP
gcs_output_uri_prefix = f"{BUCKET_URI}/{display_name}"

# Uses GPU PyTorch image for training
TRAIN_VERSION = "pytorch-gpu.1-11"
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)

CMDARGS = [
    f"--epochs={EPOCHS}",
    "--arch=resnet18",
    f"--batch-size={BATCH_SIZE}",
    "--dist-backend=nccl",
    f"--data=/gcs/{DATA_BUCKET_NAME}/tiny-imagenet-200",
    f"--workers={NUM_CPUS}",
    f"--ngpus-per-node={TRAIN_NGPU}"
]

job = aiplatform.CustomTrainingJob(
    display_name=display_name,
    script_path="trainer-gpu/main.py",
    container_uri=TRAIN_IMAGE,
    staging_bucket=gcs_output_uri_prefix,
    requirements=["tensorboard==2.5.0"],
#    model_serving_container_image_uri=DEPLOY_IMAGE,
)

# Start the training
model = job.run(
        args=CMDARGS,
        replica_count=REPLICAS + 1,
        machine_type=TRAIN_COMPUTE,
        tensorboard=TENSORBOARD_NAME,
        service_account=SERVICE_ACCOUNT,
        sync=True,
        accelerator_type=TRAIN_GPU,
        accelerator_count=TRAIN_NGPU,
    )

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete artifact repository
! gcloud artifacts repositories delete $REPOSITORY --location=$REGION --quiet

# Delete Tensorboard instance
# tensorboard.delete
! gcloud ai tensorboards delete $TENSORBOARD_NAME

# Delete Cloud Storage objects that were created
! gsutil rm -rf $BUCKET_URI/content_name

if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI